### Разработка нейросетевых функций. Операция Convolution Transpose

#### Описание параметров

in_channels (int) – Number of channels in the input image\
out_channels (int) – Number of channels produced by the convolution\
kernel_size (int or tuple) – Size of the convolving kernel\
stride (int or tuple, optional) – Stride of the convolution. Default: 1\
padding (int, tuple or str, optional) – Padding added to all four sides of the input. Default: 0\
padding_mode (str, optional) – 'zeros', 'reflect', 'replicate' or 'circular'. Default: 'zeros'\
dilation (int or tuple, optional) – Spacing between kernel elements. Default: 1\
groups (int, optional) – Number of blocked connections from input channels to output channels. Default: 1\
bias (bool, optional) – If True, adds a learnable bias to the output. Default: True\

In [6]:
import torch
import numpy as np
import warnings
warnings.simplefilter("ignore")

def myConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0, output_padding=0, dilation=1, bias=True, padding_mode='zeros'):
    def Conv(matrix):
        if bias == True:
            bias_val = torch.rand(out_channels)
        else:
            bias_val = torch.zeros(out_channels)

        if (padding_mode != 'zeros'):
            raise Exception('Ошибка. padding_mode задан неверно!')

        if type(kernel_size) == tuple:
            filter = torch.rand(in_channels, out_channels, kernel_size[0], kernel_size[1])
        if type(kernel_size) == int:
            filter = torch.rand(in_channels, out_channels, kernel_size, kernel_size)

        li = []

        for l in range(out_channels):
            feature_map = torch.zeros((matrix.shape[1]-1)*stride + dilation * (kernel_size-1)+1, (matrix.shape[2]-1)
                                      *stride  + dilation * (kernel_size-1)+1 )
            for c in range (in_channels):
                for i in range (0, matrix.shape[1]):  
                    for j in range (0, matrix.shape[2]):
                        val = matrix[c][i][j]
                        proizv = val*filter[c][l]
                        zero_tensor = torch.zeros((filter.shape[2]-1)*dilation+1, (filter.shape[3]-1)*dilation+1)

                        for a in range (0, zero_tensor.shape[0], dilation):
                            for b in range (0, zero_tensor.shape[1], dilation):
                                zero_tensor[a][b] = proizv[a//dilation][b//dilation]

                        res = np.add((zero_tensor), feature_map[i*stride:i*stride+(filter.shape[2]-1)*
                                                                dilation+1, j*stride:j*stride+(filter.shape[3]-1)*dilation+1])
                        feature_map[i*stride:i*stride+(filter.shape[2]-1)*dilation+1, j*stride:j*stride
                                    +(filter.shape[3]-1)*dilation+1] = res


            li.append(np.add(feature_map, np.full((feature_map.shape), bias_val[l])))


        for t in range(len(li)):
            if output_padding > 0:
                pad_func = torch.nn.ConstantPad1d((0, output_padding, 0, output_padding), 0)
                li[t] = pad_func(li[t])

            li[t] = li[t][0+padding:li[t].shape[0]-padding, 0+padding:li[t].shape[1]-padding]


        return li, filter, torch.tensor(bias_val)

    return Conv

In [11]:
img1 = torch.rand(8, 5, 6)

Func = myConvTranspose2d(in_channels=8, out_channels=2, kernel_size=3, stride=1, padding=0, output_padding=0,
                         dilation=1, bias=True, padding_mode='zeros')
result, kernel, bias_val = Func(img1)
torchFunction = torch.nn.ConvTranspose2d(in_channels=8, out_channels=2, kernel_size=3, stride=1, padding=0,
                                         output_padding=0, dilation=1, bias=True, padding_mode='zeros')
torchFunction.weight.data = kernel
torchFunction.bias.data = bias_val

print(result);
print(end='\n\n\n\n')
print(str(np.array(torchFunction(img1).data)))

[tensor([[ 3.1331,  6.5988,  7.8433,  6.5570,  6.3482,  7.2038,  4.6291,  2.1720],
        [ 5.4214, 10.5602, 14.8310, 14.6792, 14.4514, 13.5618,  8.8346,  4.7122],
        [ 8.6316, 14.2901, 20.4837, 19.3354, 19.2887, 20.2009, 12.1302,  7.1173],
        [ 9.2375, 14.3471, 21.5667, 19.9796, 20.3780, 20.4666, 13.0241,  5.8250],
        [ 9.0618, 13.9048, 20.3571, 18.3469, 20.3709, 18.7933, 13.9800,  5.8660],
        [ 5.4941,  7.8956, 13.7685, 12.0230, 14.0983, 11.3043,  8.5271,  4.8131],
        [ 2.6840,  3.4864,  5.6875,  6.1463,  5.8653,  6.0680,  3.1185,  2.1511]]), tensor([[ 2.6149,  4.8761,  7.0615,  6.8501,  6.2363,  6.9232,  5.1690,  3.8666],
        [ 4.5600,  7.8935, 12.2280, 13.1924, 12.5405, 12.4767,  7.9589,  5.8672],
        [ 7.4190, 11.0547, 17.7375, 18.3442, 17.2785, 16.8891, 12.1377,  6.7916],
        [ 7.2754, 11.4281, 18.6729, 18.1914, 19.9586, 18.1694, 12.9631,  6.3629],
        [ 7.7623, 10.4667, 17.8200, 15.5612, 18.8985, 16.8510, 12.5185,  6.4444],
        [ 5.1

In [14]:
img2 = torch.rand(3, 28, 28)

Func2 = myConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,
                          output_padding=0, dilation=3, bias=True, padding_mode='zeros')
result, kernel, bias_val = Func2(img2)
torchFunction2 = torch.nn.ConvTranspose2d(in_channels=3, out_channels=2, kernel_size=3, stride=10, padding=0,
                                          output_padding=0, dilation=3, bias=True, padding_mode='zeros')
torchFunction2.weight.data = kernel
torchFunction2.bias.data = bias_val

print(result);
print(end='\n\n\n\n')
print(str(np.array(torchFunction2(img2).data)))
     

[tensor([[1.4267, 0.8546, 0.8546,  ..., 0.8546, 0.8546, 1.3524],
        [0.8546, 0.8546, 0.8546,  ..., 0.8546, 0.8546, 0.8546],
        [0.8546, 0.8546, 0.8546,  ..., 0.8546, 0.8546, 0.8546],
        ...,
        [0.8546, 0.8546, 0.8546,  ..., 0.8546, 0.8546, 0.8546],
        [0.8546, 0.8546, 0.8546,  ..., 0.8546, 0.8546, 0.8546],
        [1.0064, 0.8546, 0.8546,  ..., 0.8546, 0.8546, 1.0848]]), tensor([[0.6911, 0.2651, 0.2651,  ..., 0.2651, 0.2651, 1.1186],
        [0.2651, 0.2651, 0.2651,  ..., 0.2651, 0.2651, 0.2651],
        [0.2651, 0.2651, 0.2651,  ..., 0.2651, 0.2651, 0.2651],
        ...,
        [0.2651, 0.2651, 0.2651,  ..., 0.2651, 0.2651, 0.2651],
        [0.2651, 0.2651, 0.2651,  ..., 0.2651, 0.2651, 0.2651],
        [0.4774, 0.2651, 0.2651,  ..., 0.2651, 0.2651, 0.9208]])]




[[[1.42675    0.85457486 0.85457486 ... 0.85457486 0.85457486 1.3524214 ]
  [0.85457486 0.85457486 0.85457486 ... 0.85457486 0.85457486 0.85457486]
  [0.85457486 0.85457486 0.85457486 ... 0.8545748

In [16]:
img3 = torch.rand(5, 6, 6)

Func3 = myConvTranspose2d(in_channels=5, out_channels=1, kernel_size=3, stride=3, padding=5,
                          output_padding=2, dilation=1, bias=True, padding_mode='zeros')
result, kernel, bias_val = Func3(img3)
torchFunction3 = torch.nn.ConvTranspose2d(in_channels=5, out_channels=1, kernel_size=3, stride=3,
                                          padding=5, output_padding=2, dilation=1, bias=True, padding_mode='zeros')
torchFunction3.weight.data = kernel
torchFunction3.bias.data = bias_val

print(result);
print(end='\n\n\n\n')
print(str(np.array(torchFunction3(img3).data)))

[tensor([[1.4245, 0.8285, 0.6493, 0.8031, 1.0417, 1.2434, 1.4128, 0.9784, 0.8634,
         1.2265],
        [1.7626, 1.6280, 1.6336, 1.3068, 1.5904, 1.4299, 1.3380, 1.2563, 1.3884,
         1.3867],
        [1.0492, 1.1578, 1.8666, 0.6443, 1.6881, 1.8220, 0.9544, 0.7994, 1.4644,
         0.6034],
        [1.5859, 1.3923, 1.0404, 1.5442, 1.4546, 1.4025, 1.1356, 1.0652, 0.7482,
         1.0958],
        [1.7674, 1.1218, 1.2143, 1.1004, 2.0716, 2.3750, 1.8421, 1.7587, 1.9214,
         1.6972],
        [0.7710, 1.4819, 1.8958, 0.3642, 1.6712, 2.2249, 1.1251, 1.4839, 1.8148,
         1.1210],
        [1.8679, 1.0996, 1.8168, 1.6252, 1.7639, 1.4604, 2.0899, 1.5154, 1.1752,
         1.4218],
        [1.0385, 1.0927, 1.3357, 1.1244, 1.6068, 1.9509, 1.6546, 1.6209, 1.2246,
         1.1433],
        [0.3844, 0.8349, 1.5216, 0.3429, 1.4247, 1.9245, 0.8721, 1.4698, 1.9388,
         0.6730],
        [1.0597, 0.9510, 1.0465, 1.4904, 1.3996, 1.4255, 1.7996, 1.4678, 1.2180,
         1.0920]])]




[[[